In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

# ----------------------------------------------------------
# 1. Load dataset
# ----------------------------------------------------------
df = pd.read_csv('/Users/adamtaylor/Documents/Personal/Finance/StockDB/yfinGens/Data1.2/stock_metrics_120325.csv')

# ----------------------------------------------------------
# 2. Define future safety as negative beta 
#    (low beta = safer)
# ----------------------------------------------------------
df['future_safety'] = -df['beta_1y']

# ----------------------------------------------------------
# 3. Clean the data
# ----------------------------------------------------------
df = df.replace([np.inf, -np.inf], np.nan)

needed_cols = ['sharpe_1y', 'vol_z', 'future_safety']
df_clean = df.dropna(subset=needed_cols).copy()

print(f"Rows before cleaning: {len(df)}")
print(f"Rows after cleaning:  {len(df_clean)}\n")

# ----------------------------------------------------------
# 4. Standardize the predictors for scale-free regression
# ----------------------------------------------------------
for col in ['sharpe_1y', 'vol_z']:
    mu = df_clean[col].mean()
    sigma = df_clean[col].std(ddof=0)
    df_clean[col + '_z'] = (df_clean[col] - mu) / sigma

# ----------------------------------------------------------
# 5. Regression to derive optimal safety-score weights
#    future_safety ~ sharpe_1y_z + vol_z_z
# ----------------------------------------------------------
Xz = df_clean[['sharpe_1y_z', 'vol_z_z']]
Xz = sm.add_constant(Xz)
y = df_clean['future_safety']

model_z = sm.OLS(y, Xz).fit()
print("=== Standardized regression: future_safety ~ sharpe_1y_z + vol_z_z ===")
print(model_z.summary())

# Extract coefficients
coef_sharpe = model_z.params['sharpe_1y_z']
coef_volz   = model_z.params['vol_z_z']

print("\nExact regression-derived weights:")
print(f"Sharpe_z coefficient: {coef_sharpe:.4f}")
print(f"Vol_z_z coefficient:  {coef_volz:.4f}")

# ----------------------------------------------------------
# 6. Construct the NEW optimized safety score
# ----------------------------------------------------------
df_clean['beta_safety_score_exact'] = (
    coef_sharpe * df_clean['sharpe_1y_z'] +
    coef_volz   * df_clean['vol_z_z']
)

# ----------------------------------------------------------
# 7. Test how well the NEW score predicts safety
# ----------------------------------------------------------
X_new = sm.add_constant(df_clean['beta_safety_score_exact'])
model_new = sm.OLS(df_clean['future_safety'], X_new).fit()

print("\n=== Regression: future_safety ~ NEW beta_safety_score_exact ===")
print(model_new.summary())

# ----------------------------------------------------------
# 8. Bonferroni correction (optional)
# ----------------------------------------------------------
pvals = list(model_z.pvalues.values) + list(model_new.pvalues.values)
names = (
    list(model_z.pvalues.index) +
    ["new_" + n for n in model_new.pvalues.index]
)

reject, pvals_corr, _, _ = multipletests(pvals, alpha=0.05, method='bonferroni')

print("\n=== Bonferroni-corrected p-values ===")
for name, p in zip(names, pvals_corr):
    print(f"{name:25s} corrected p = {p:.6f}")


Rows before cleaning: 196
Rows after cleaning:  195

=== Standardized regression: future_safety ~ sharpe_1y_z + vol_z_z ===
                            OLS Regression Results                            
Dep. Variable:          future_safety   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     79.09
Date:                Sat, 06 Dec 2025   Prob (F-statistic):           8.79e-26
Time:                        21:28:23   Log-Likelihood:                -145.72
No. Observations:                 195   AIC:                             297.4
Df Residuals:                     192   BIC:                             307.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------

In [10]:
import numpy as np
import statsmodels.api as sm

N_BOOT = 10000

coef_sharpe_list = []
coef_volz_list   = []
r2_list          = []

dfb = df_clean[['sharpe_1y_z', 'vol_z_z', 'future_safety']].dropna()

n = len(dfb)

for _ in range(N_BOOT):
    # sample with replacement
    sample = dfb.sample(n, replace=True)

    X = sm.add_constant(sample[['sharpe_1y_z', 'vol_z_z']])
    y = sample['future_safety']

    model = sm.OLS(y, X).fit()

    # store coefficients
    coef_sharpe_list.append(model.params['sharpe_1y_z'])
    coef_volz_list.append(model.params['vol_z_z'])
    r2_list.append(model.rsquared)

coef_sharpe_arr = np.array(coef_sharpe_list)
coef_volz_arr   = np.array(coef_volz_list)
r2_arr          = np.array(r2_list)

print("=== Bootstrap Results (10,000 runs) ===")
print(f"Sharpe coef mean: {coef_sharpe_arr.mean():.4f}, std: {coef_sharpe_arr.std():.4f}")
print(f"Vol_z coef mean:  {coef_volz_arr.mean():.4f}, std: {coef_volz_arr.std():.4f}")
print(f"R2 mean:          {r2_arr.mean():.4f}, std: {r2_arr.std():.4f}")
print(f"Sharpe sign flips: {np.mean(coef_sharpe_arr > 0):.3f} (fraction of flips)")


=== Bootstrap Results (10,000 runs) ===
Sharpe coef mean: -0.1469, std: 0.0348
Vol_z coef mean:  0.4170, std: 0.0685
R2 mean:          0.4584, std: 0.0498
Sharpe sign flips: 0.000 (fraction of flips)
